In [2]:
import random
import collections

def count_values(input_list,num_walkers):
    counter = collections.Counter(input_list)
    return sorted((val, counter[val]/num_walkers) for val in set(input_list))

def random_walk(num_steps=4, start=0, num_walkers=10000):
    """
    1-Dimensional Random Walk simulation.

    :param num_steps: The number of steps to simulate (default: 100)
    :param start: Starting position (default: 0)
    :return: A list containing the random walk's coordinates in the format [x, index].
    """

    # Initialize walk with starting position
    walk = [[ start for _ in range(num_walkers)]+[0]]

    for k in range(1, num_steps + 1):
        walk.append([k])
        for walker in range(num_walkers+1):
            # Append new position (x and index) to walk
            walk[-1].append(walk[-2][walker] + random.choice([-1, 1]))

    return walk

# Example usage:
num_walkers=10000
for step in random_walk(4,0,num_walkers):
    print(step[0],count_values(step[1:],num_walkers))

0 [(0, 1.0)]
1 [(-1, 0.501), (1, 0.4991)]
2 [(-2, 0.2446), (0, 0.5048), (2, 0.2507)]
3 [(-3, 0.1198), (-1, 0.377), (1, 0.3797), (3, 0.1236)]
4 [(-4, 0.0614), (-2, 0.2449), (0, 0.3839), (2, 0.2475), (4, 0.0624)]
